In [1]:
!pip install PyPDF2 langchain langchain_community langchain_groq faiss_cpu huggingface_hub langchain_groq torch
!pip install InstructorEmbedding sentence-transformers==2.2.2

In [2]:
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_groq import ChatGroq

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [3]:
groq_api_key = "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"

# Extraction of text from PDFs
def get_pdf_text(pdf_paths):
    text = ""
    for pdf_path in pdf_paths:
        try:
            pdf_reader = PdfReader(pdf_path)
            for page in pdf_reader.pages:
                text += page.extract_text() or ""
        except Exception as e:
            print(f"Error reading PDF {pdf_path}: {e}")
    return text

In [4]:
# Dividing the raw text into different chunks
def get_text_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1000,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

In [5]:
# Creating a vector store embeddings from HuggingFace
def get_vectorstore(text_chunks):
    if not text_chunks:
        print("No text chunks to create embeddings.")
        return None
    embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    vectorstore = FAISS.from_texts(texts=text_chunks, embedding=embeddings)
    return vectorstore

In [6]:
# Creating a conversation chain to store the context for follow-up questions
def get_conversation_chain(vectorstore, groq_api_key):
    llm = ChatGroq(groq_api_key=groq_api_key, model_name="mixtral-8x7b-32768")
    memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

In [7]:
# Handling the user input
def handle_userinput(conversation_chain, user_question):
    response = conversation_chain({'question': user_question})
    chat_history = response['chat_history']

    for i, message in enumerate(chat_history):
        if i % 2 == 0:
            print(f"User: {message.content}")
        else:
            print(f"Bot: {message.content}")

In [8]:
def main():
        print("Welcome to the File Processor!")

        pdf_paths = input("Enter the paths to PDF files (separated by commas): ").split(',')
        pdf_paths = [path.strip() for path in pdf_paths]
        raw_text = get_pdf_text(pdf_paths)

        # Get text chunks
        text_chunks = get_text_chunks(raw_text)

        # Create vector store with embeddings
        vectorstore = get_vectorstore(text_chunks)

        if vectorstore:
            # Create conversation chain
            conversation_chain = get_conversation_chain(vectorstore, groq_api_key)

            while True:
                user_question = input("Ask something (type 'exit' to quit): ")
                if user_question.lower() in ['exit' , 'quit']:
                    break
                handle_userinput(conversation_chain, user_question)

if __name__ == '__main__':
    main()

Welcome to the File Processor!
Enter the paths to PDF files (separated by commas): /content/sodapdf-converted (1).pdf


/usr/local/lib/python3.10/dist-packages/InstructorEmbedding/instructor.py:7: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'cached_download' (from 'huggingface_hub.file_download') is deprecated and will be removed from version '0.26'. Use `hf_h

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/270 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

load INSTRUCTOR_Transformer


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/models/Dense.py:63: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no lon

max_seq_length  512
Ask something (type 'exit' to quit): provide me the content you get


<ipython-input-7-bf4f56e51d98>:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response = conversation_chain({'question': user_question})


User: provide me the content you get
Bot: Based on the context provided, it seems like Taylor is going to provide Alex with some content. However, I don't have the subsequent context to know what specific content that is. I can only answer questions directly related to the text provided.
Ask something (type 'exit' to quit): you know anything about llm
User: provide me the content you get
Bot: Based on the context provided, it seems like Taylor is going to provide Alex with some content. However, I don't have the subsequent context to know what specific content that is. I can only answer questions directly related to the text provided.
User: you know anything about llm
Bot: Based on the information provided, it seems that Alex asked about an LLM program or degree after Taylor said they would see each other around. However, Taylor's response did not include any information about an LLM program or degree. An LLM is a Master of Laws degree, which is a postgraduate law degree that is usuall